In [ ]:
! pip install -r ../../requirements.txt
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [2]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset('/home/sas/GitRepos/AlexaGPT2/api/ML/train.txt','/home/sas/GitRepos/AlexaGPT2/api/ML/test.txt',tokenizer)

/home/sas/.local/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  warnings.warn(


In [9]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("./model_parameters")


training_args = TrainingArguments(
    output_dir="/run/media/Data/model_parameters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=24, # batch size for training
    per_device_eval_batch_size=24, # batch size for evaluation
    eval_steps = 200, # Number of update steps between two evaluations.
    save_steps=400, # after # steps model is saved 
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [10]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=60, training_loss=3.7023585001627604, metrics={'train_runtime': 18.516, 'train_samples_per_second': 3.24, 'total_flos': 134179960651776, 'epoch': 3.0})

In [11]:
trainer.save_model()

In [1]:
from transformers import pipeline

generator = pipeline('text-generation',model="/run/media/Data/model_parameters", tokenizer='gpt2', config={'max_length':10000})


In [17]:
def generate_response(input):
    print(input)
    return generator(input, do_sample=True, max_length=len(input.split())+60, top_k=50, top_p=0.95, num_returned_sequences=1)[0]['generated_text']

print(generate_response(""))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



For a price
There is no shortage of joy
In the night I climb high and high
Of pain and pain
In the night I stand upright
When all my troubles are gone
And the love soothes my soul
But it's too late to go
I must fight

